# Transform a positive ZZ^m-grading such that it has positive entries only

We are given generators $r_1,\ldots, r_n$ in $ZZ^m$ of some grading on a polynomial ring such that the grading is positive, i.e. the degree 0 part has finite dimension. We want the $r_i$ to have positive entries only.

In [1]:
using Oscar


You've found 
DORY
Version 0.1 ... 
 ... which comes with absolutely no warranty whatsoever
(c) 2019 by Avinash Kulkarni


You've found 
DORY
Version 0.1 ... 
 ... which comes with absolutely no warranty whatsoever
(c) 2019 by Avinash Kulkarni

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.8.2-DEV ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


┌ Info: Precompiling Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13]
└ @ Base loading.jl:1423
┌ Warning: Replacing docs for `Oscar.F :: Union{Tuple{Hecke.SRow{T}}, Tuple{T}} where T` in module `Oscar`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `Oscar.F :: Union{Tuple{Vector{T}}, Tuple{T}} where T` in module `Oscar`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Package Oscar does not have pAdicSolver in its dependencies:
│ - If you have Oscar checked out for development and have
│   added pAdicSolver as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Oscar
└ Loading pAdicSolver into Oscar from project dependency, future warnings for Oscar are suppressed.
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



We start by doing an example:

In [2]:
rs = [-1 0 0; -1 -1 0; -1 -1 -1; -1 0 -1]

4×3 Matrix{Int64}:
 -1   0   0
 -1  -1   0
 -1  -1  -1
 -1   0  -1

First we compute the cone $C$ that is spanned by these generators:

In [3]:
C = positive_hull(rs)

A polyhedral cone in ambient dimension 3

Now we check that it is fulldimensional, we will stay with the full-dimensional case for now:

In [8]:
@assert isfulldimensional(C) "Cone spanned by generator degrees needs to be full-dimensional"

Now we want to find a simplicial cone containing $C$. This can be done by choosing a subset of the facets which has full rank. The facet vectors have full rank because the cone is fulldimensional.

In [29]:
F = linear_inequality_matrix(facets(C))

[1   -1    0]
[1    0   -1]
[0    1    0]
[0    0    1]

In [39]:
index = 2
full_rank_subset = [1]
full_rank = rank(F)
current_rank = rank(F[full_rank_subset, :])
while current_rank < full_rank
    for i in index:nrows(F)
        test = Vector{Int}(full_rank_subset)
        append!(test, i)
        testrank = rank(F[test,:])
        if testrank > current_rank
            index = i+1
            current_rank = testrank
            append!(full_rank_subset, i)
            break
        end
    end
end
print(full_rank_subset)

[1, 2, 3]

We build the associated cone from these inequalities and computes its rays.

In [71]:
Csimplicial = cone_from_inequalities(F[full_rank_subset,:])

A polyhedral cone in ambient dimension 3

In [72]:
rays(Csimplicial)

3-element SubObjectIterator{RayVector{Polymake.Rational}}:
 [-1, -1, -1]
 [-1, 0, -1]
 [0, 0, 1]

In [73]:
ispointed(Csimplicial)

true

In [45]:
Polymake.polytope.included_polyhedra(C.pm_cone, Csimplicial.pm_cone)

true

Now we can use the Hermite normal form for finding a coordinate change that brings `Csimplicial` into the positive orthant while preserving the lattice. 

In [50]:
CsRays = Polymake.common.primitive(Csimplicial.pm_cone.RAYS)
CsRays = matrix(ZZ, CsRays)
nf = AbstractAlgebra.hnf_with_transform(transpose(CsRays))

([1 0 0; 0 1 0; 0 0 1], [0 -1 0; -1 1 0; -1 0 1])

In [52]:
CsRays_transformed = transpose(nf[1])

[1   0   0]
[0   1   0]
[0   0   1]

In [53]:
transformation = transpose(nf[2])

[ 0   -1   -1]
[-1    1    0]
[ 0    0    1]

In [61]:
@assert CsRays * transformation == CsRays_transformed "Maybe order of transformation is wrong?"

In [58]:
original = matrix(ZZ, rs)

[-1    0    0]
[-1   -1    0]
[-1   -1   -1]
[-1    0   -1]

In [59]:
original * transformation

[0   1   1]
[1   0   1]
[1   0   0]
[0   1   0]

# As one function

In [64]:
function transform_to_positive_orthant(rs::Matrix{Int})
    C = positive_hull(rs)
    @assert isfulldimensional(C) "Cone spanned by generator degrees needs to be full-dimensional"
    F = linear_inequality_matrix(facets(C))
    
    # Find a simplicial cone containing C
    index = 2
    full_rank_subset = [1]
    full_rank = rank(F)
    current_rank = rank(F[full_rank_subset, :])
    while current_rank < full_rank
        for i in index:nrows(F)
            test = Vector{Int}(full_rank_subset)
            append!(test, i)
            testrank = rank(F[test,:])
            if testrank > current_rank
                index = i+1
                current_rank = testrank
                append!(full_rank_subset, i)
                break
            end
        end
    end
    Csimplicial = cone_from_inequalities(F[full_rank_subset,:])
    
    @assert Polymake.polytope.included_polyhedra(C.pm_cone, Csimplicial.pm_cone) "Cone containment violated"
    CsRays = Polymake.common.primitive(Csimplicial.pm_cone.RAYS)
    CsRays = matrix(ZZ, CsRays)
    nf = AbstractAlgebra.hnf_with_transform(transpose(CsRays))
    CsRays_transformed = transpose(nf[1])
    transformation = transpose(nf[2])
    @assert CsRays * transformation == CsRays_transformed "Maybe order of transformation is wrong?"
    original = matrix(ZZ, rs)
    return original * transformation, transformation
end

transform_to_positive_orthant (generic function with 1 method)

In [67]:
t_rs = transform_to_positive_orthant(rs)

([0 1 1; 1 0 1; 1 0 0; 0 1 0], [0 -1 -1; -1 1 0; 0 0 1])

In [68]:
t_rs[1]

[0   1   1]
[1   0   1]
[1   0   0]
[0   1   0]

In [69]:
hexagon = [-1 0 0; -1 1 0; -1 2 1; -1 2 2; -1 1 2; -1 0 1]
t_hexagon = transform_to_positive_orthant(hexagon)

([0 0 1; 1 0 2; 2 1 2; 2 2 1; 1 2 0; 0 1 0], [0 0 -1; 1 0 1; 0 1 -1])

In [70]:
inv(t_hexagon[2])

[ 1   1   0]
[-1   0   1]
[-1   0   0]